In [ ]:
# Template

In [ ]:
import visa
import numpy as np
import pylab as pl
import warnings
import time
from datetime import datetime
#from Ipython.html.widgets import interact
from IPython.display import display
from IPython.display import HTML, Image, Latex, Math, Markdown, SVG
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf')

plt.rcParams['savefig.dpi'] = 75

plt.rcParams['figure.autolayout'] = False
plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2.0
plt.rcParams['lines.markersize'] = 8
plt.rcParams['legend.fontsize'] = 14

plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = "serif"
plt.rcParams['font.serif'] = "cm"
plt.rcParams['text.latex.preamble'] = r"\usepackage{subdepth}, \usepackage{type1cm}"

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')


In [ ]:
# Test configuration
cycle_time = 4
sleep_time = 2 # sleep for X seconds in each cycle
repeat_counter = 1
log_data = datetime.now().strftime('%H_%M_%d_%m_%Y')

In [ ]:
def oscilloscope_screenshot(repeat_counter, scope, port, interface, log_data):
    scope.encoding = 'ascii' # default 'ascii'
    scope.read_termination = None # default None
    scope.write_termination = '\r\n' # default '\r\n'
    
    vc = visa.constants # Alias for easier reading
    bufferSize = 1024

    # Serial Port Settings
    baudRate = 9600
    flowCtrl = vc.VI_ASRL_FLOW_RTS_CTS
    #flowCtrl = vc.VI_ASRL_FLOW_NONE
    #==============================================

    if interface == vc.VI_INTF_ASRL:
        lib.set_attribute(scope.session, vc.VI_ATTR_ASRL_FLOW_CNTRL, flowCtrl)
        lib.set_attribute(scope.session, vc.VI_ATTR_ASRL_BAUD, baudRate)
        port = 'RS232'
        startDelaySec = 1
    elif interface == vc.VI_INTF_USB:
        port = 'USB'
        startDelaySec = 2
    elif interface == vc.VI_INTF_GPIB:
        port = 'GPIB'
        startDelaySec = 5

    # Date information
    start_time = datetime.now()
    print('Recorded on {}'.format(start_time))

    fileSaveLocation = r'C:\Users\ahamidi\version-control\Test\ScreenShot{}_{}.png'.format(repeat_counter,log_data)
    
    #![title](fileSaveLocation)
    
    print(scope.query("*IDN?"))
    scope.write("HARDCOPY:FORMAT PNG")
    scope.write("HARDCOPY:LAYOUT PORTRait")
    scope.write("HARDCOPY:PORT " + port)
    scope.write("HARDCOPY START")

    print("Starting Transfering the screenshot in: ", end = "")
    for x in range(0, startDelaySec):
        print(str.format("{0}...", startDelaySec - x), end = "")
        time.sleep(1)
    print("Now!")

    # If using RS-232 then need to disable end on termination character while reading the image data
    if lib.get_attribute(scope.session, vc.VI_ATTR_INTF_TYPE)[0] == vc.VI_INTF_ASRL:
        lib.set_attribute(scope.session, vc.VI_ATTR_ASRL_END_IN, 0)

    # Read the BMP header bytes and extract the file size
    warnings.filterwarnings("ignore", category=Warning) #The read will produce a VI_SUCCESS_MAX_CNT warning so suppress
    imgBytes = lib.read(scope.session, 14)[0]
    lengthBytes = imgBytes[2:6]
    fileSize = int.from_bytes(lengthBytes, byteorder='little', signed=False)
    bytesLeft = fileSize - 14

    # Read the rest of the image
    while bytesLeft > 0:
        imgBytes = imgBytes + lib.read(scope.session, bufferSize)[0]
        bytesLeft = bytesLeft - bufferSize

        if bytesLeft < bufferSize:
            imgBytes = imgBytes + lib.read(scope.session, bytesLeft)[0]
            bytesLeft = 0

    # Re-enable end on termination character for RS-232
    if lib.get_attribute(scope.session, vc.VI_ATTR_INTF_TYPE)[0] == vc.VI_INTF_ASRL:
        lib.set_attribute(scope.session, vc.VI_ATTR_ASRL_END_IN, 1)

    # Save the bytes to a file
    imgFile = open(fileSaveLocation, "wb")
    imgFile.write(imgBytes)
    imgFile.close()

    print("Transfer Completed!")
    print("Image saved to " + fileSaveLocation)
    img=Image(filename = fileSaveLocation, format ='jpeg', width = 400,height = 400)
    display(img)


In [ ]:
def waveform_recorder(repeat_counter, scope, port, interface , log_data):
    
    scope.encoding = 'latin_1' # default 'ascii'
    scope.read_termination = '\n' # default None
    scope.write_termination = None # default '\r\n'
    
    t1 = time.perf_counter()
    r = scope.query('*opc?') # sync
    t2 = time.perf_counter()
    print('reset time: {} s'.format(t2 - t1))

    # io config
    scope.write('header 0')
    scope.write('data:encdg RIBINARY')
    scope.write('data:source CH2') # channel 2 :chirp signal
    scope.write('data:start 1') # first sample
    record = int(scope.query('wfmpre:nr_pt?'))
    scope.write('data:stop {}'.format(record)) # last sample
    scope.write('wfmpre:byt_nr 1') # 1 byte per sample

    # acq config
    scope.write('acquire:state stop') # (equivalent to pressing) STOP (button)
    scope.write('acquire:stopafter SEQUENCE') # single 
    scope.write('acquire:state ON') # (equivalent to pressing) RUN (button)
    scope.write('acquire:stopafter runstop')

    
    t5 = time.perf_counter()
    r = scope.query('*opc?') # sync
    t6 = time.perf_counter()
    print('acquire time: {} s'.format(t6 - t5))

#     r = scope.write('*WAI') # sync
#     print("sync after run command {}".format(r))
    
    # data query
    t7 = time.perf_counter()
    bin_wave = scope.query_binary_values('curve?', datatype='b', container=np.array)
    t8 = time.perf_counter()
    print('transfer time: {} s'.format(t8 - t7))

#     scope.write('acquire:state run') # (equivalent to pressing) RUN (button)
#     run_button = scope.query('acquire:state?')
#     print('Run button is {} after data query'. format(run_button))
    
    # retrieve scaling factors
    tscale = float(scope.query('wfmpre:xincr?'))
    tstart = float(scope.query('wfmpre:xzero?'))
    vscale = float(scope.query('wfmpre:ymult?')) # volts / level
    voff = float(scope.query('wfmpre:yzero?')) # reference voltage
    vpos = float(scope.query('wfmpre:yoff?')) # reference position (level)

    # error checking
    r = int(scope.query('*esr?'))
    print('event status register: 0b{:08b}'.format(r))
    r = scope.query('allev?').strip()
    print('all event messages: {}'.format(r))
    
#     scope.write('acquire:state run') # (equivalent to pressing) RUN (button)
#     run_button = scope.query('acquire:state?')
#     print('Run button is {} aftererror checking'. format(run_button))
    
    # create scaled vectors
    # horizontal (time)
    total_time = tscale * record
    tstop = tstart + total_time
    scaled_time = np.linspace(tstart, tstop, num=record, endpoint=False)
    # vertical (voltage)
    unscaled_wave = np.array(bin_wave, dtype='double') # data type conversion
    scaled_wave = (unscaled_wave - vpos) * vscale + voff
    waveSaveLocation = r'C:\Users\ahamidi\version-control\Test\wave_form{}_{}'.format(repeat_counter,log_data)
    np.save(waveSaveLocation,scaled_wave)

    # plotting
#     pl.step(scaled_time, scaled_wave)
    pl.plot(scaled_time, scaled_wave)
    pl.title('channel 1') # plot label
    pl.xlabel('time (seconds)') # x label
    pl.ylabel('voltage (volts)') # y label
    print("\nlook for plot window...")
    pl.show()
    pl.savefig('plot{}_{}.png'.format(repeat_counter,log_data), bbox_inches='tight')

    

In [ ]:
rm = visa.ResourceManager()
lib = rm.visalib
vc = visa.constants
scope = rm.open_resource('GPIB0::20::INSTR')
interface = lib.get_attribute(scope.session, vc.VI_ATTR_INTF_TYPE)[0]

if ( interface != 1):
    print('Invalid port. Please ente')
else : 
    # initialize the port and scope session
    port ='GPIB'
    scope.timeout = 10000
    scope.set_visa_attribute(vc.VI_ATTR_IO_PROT,3)
    scope.write('wfmpre:nr_pt 10000')
    print(scope.interface_type)
    print("Port:  " + port)
    print("Device Query: " + scope.query('*IDN?'))
    print("Device timeout: {}".format(scope.timeout))
    print("Session ID: {} ".format(scope.session))

In [ ]:
while(repeat_counter < cycle_time):
    oscilloscope_screenshot(repeat_counter, scope, port, interface,log_data)
    scope.write('*cls')
    waveform_recorder(repeat_counter, scope, port, interface, log_data)     
    repeat_counter += 1
    scope.write('*cls')
    time.sleep(sleep_time)


In [ ]:
scope.close()
rm.close()